Ejemplo de lectura de bases de datos sql con pandas

# Importamos las librerias necesarias e instalamos la base de datos en la nube

In [1]:
# Imports necesarios

import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd

Descargamos la base de datos almacenada en la nube con *wget*.

wget es una herramienta que nos va a permitir descargar contenido a partir de servidores web. Utilizamos la flag -P para indicar que es una URL preferente.

In [2]:
!wget -P data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/classic_rock.db

--2023-05-12 00:52:28--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/classic_rock.db
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5652480 (5.4M) [binary/octet-stream]
Saving to: ‘data/classic_rock.db.2’

classic_rock.db.2   100%[===================>]   5.39M   503KB/s    in 10s     

2023-05-12 00:52:39 (550 KB/s) - ‘data/classic_rock.db.2’ saved [5652480/5652480]



Inicializamos la variable `path` con nuestro archivo *classic_rock.db* recién descargado.
La variable `con` nos permite tener una conexión en vivo con la base de datos al encapsular el método `sq3.Connection(path)`

In [3]:
path = 'data/classic_rock.db'
con = sq3.Connection(path)

Llamamos nuestra conexión en tiempo real (variable con)

In [4]:
con

# Leemos la base de datos

- Declaramos la consulta de datos dentro de comillas simples.
- Usamos `SELECT *` para indicarle que seleccione todos los datos.
- Usamos `FROM` para indicarle el origen de los datos.

In [5]:
query = '''
SELECT *    
FROM rock_songs
'''

Ejecutamos nuestra consulta almacenada en query y encapsulada en observations.

El método `observations.head` nos arroja una tabla con cinco elementos que indican canción, artista, año de lanzamiento y conteo de reproducciones

In [6]:
observations = pds.read_sql(query, con)
observations.head()

,Song,Artist,Release_Year,PlayCount
0,Caught Up in You,.38 Special,1982.0,82
1,Hold On Loosely,.38 Special,1981.0,85
2,Rockin' Into the Night,.38 Special,1980.0,18
3,Art For Arts Sake,10cc,1975.0,1
4,Kryptonite,3 Doors Down,2000.0,13


Generamos una solicitud que seleccione los datos de artista, año de lanzamiento, número de canciones y promedio de reproducciones. Además, los categorizamos de forma descendente a partir del número de canciones

In [7]:
query = '''
SELECT Artist, Release_Year, Count(*) AS num_songs, AVG(PlayCount) AS avg_plays
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

# Ejecutamos
observations = pds.read_sql(query,con)
observations.head()

,Artist,Release_Year,num_songs,avg_plays
0,The Beatles,1967.0,23,6.565217
1,Led Zeppelin,1969.0,18,21.000000
2,The Beatles,1965.0,15,3.800000
3,The Beatles,1968.0,13,13.000000
4,The Beatles,1969.0,13,15.000000


# Parametros comunes

Existe una gran variedad de parametros comunes que pueden ser utilizados para leer bases de datos en SQL con formatos específicos:

- ***coerce_float***: fuerza a los números a volverse decimales
- ***parse_dates***: Modifica el formato de una columna de datos a formato de fecha
- ***chunksize***: Número de columnas que se incluiran en cada lote de información

In [8]:
query='''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

# Ejecutamos la petición
observations_generator = pds.read_sql(query,
                            con,
                            coerce_float=True, # No afecta a nuestra base de datos porque ya están en decimal (float)
                            parse_dates=['Release_Year'], # Utilizamos la columna 'Release_Year' para que tenga formato de fecha
                            chunksize=5 # Cada lote de información constará de 5 elementos
                           )

# Utilizamos un ciclo for para mostrar los lotes de información
# index será utilizado como variable iterante
# observations será la variable que contenga nuestros data sets
# enumarete es una palabra reservada que nos va a permitir obtener los valores iterados y su posición en la base de datos
# Display  nos permite ver de forma más gráfica y ordenada los valores de nuestra base de datos
for index, observations in enumerate(observations_generator):
    if index < 5:
        print(f'Observations index: {index}'.format(index))
        display(observations)

Observations index: 0


,Artist,Release_Year,num_songs,avg_plays
0,The Beatles,1970-01-01 00:32:47,23,6.565217
1,Led Zeppelin,1970-01-01 00:32:49,18,21.000000
2,The Beatles,1970-01-01 00:32:45,15,3.800000
3,The Beatles,1970-01-01 00:32:48,13,13.000000
4,The Beatles,1970-01-01 00:32:49,13,15.000000


Observations index: 1


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:50,12,13.166667
1,Led Zeppelin,1970-01-01 00:32:55,12,14.166667
2,Pink Floyd,1970-01-01 00:32:59,11,41.454545
3,Pink Floyd,1970-01-01 00:32:53,10,29.100000
4,The Doors,1970-01-01 00:32:47,10,28.900000


Observations index: 2


,Artist,Release_Year,num_songs,avg_plays
0,Fleetwood Mac,1970-01-01 00:32:57,9,35.666667
1,Jimi Hendrix,1970-01-01 00:32:47,9,24.888889
2,The Beatles,1970-01-01 00:32:43,9,2.444444
3,The Beatles,1970-01-01 00:32:44,9,3.111111
4,Elton John,1970-01-01 00:32:53,8,18.500000


Observations index: 3


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:51,8,47.750000
1,Led Zeppelin,1970-01-01 00:32:53,8,34.125000
2,Boston,1970-01-01 00:32:56,7,69.285714
3,Rolling Stones,1970-01-01 00:32:49,7,36.142857
4,Van Halen,1970-01-01 00:32:58,7,51.142857


Observations index: 4


,Artist,Release_Year,num_songs,avg_plays
0,Bruce Springsteen,1970-01-01 00:32:55,6,7.666667
1,Bruce Springsteen,1970-01-01 00:33:04,6,11.500000
2,Creedence Clearwater Revival,1970-01-01 00:32:49,6,23.833333
3,Creedence Clearwater Revival,1970-01-01 00:32:50,6,18.833333
4,Def Leppard,1970-01-01 00:33:07,6,32.000000
